In [1]:
%reset

import numpy as np
import pandas as pd

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
cvd_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CVD_Risk_Prediction Data/CVD_cleaned.csv")

In [3]:

print("Unique Values of General_Health Column: ", cvd_data['General_Health'].unique())
print("Unique Values of Checkup Column: ", cvd_data['Checkup'].unique())
print("Unique Values of Age_Category Column: ", cvd_data['Age_Category'].unique())
print("Unique Values of Exercise Column: ", cvd_data['Exercise'].unique())
print("Unique Values of Heart_Disease Column: ", cvd_data['Heart_Disease'].unique())
print("Unique Values of Skin_Cancer Column: ", cvd_data['Skin_Cancer'].unique())
print("Unique Values of Other_Cancer Column: ", cvd_data['Other_Cancer'].unique())
print("Unique Values of Depression Column: ", cvd_data['Depression'].unique())
print("Unique Values of Diabetes Column: ", cvd_data['Diabetes'].unique())
print("Unique Values of Arthritis Column: ", cvd_data['Arthritis'].unique())
print("Unique Values of Smoking_History Column: ", cvd_data['Smoking_History'].unique())

Unique Values of General_Health Column:  ['Poor' 'Very Good' 'Good' 'Fair' 'Excellent']
Unique Values of Checkup Column:  ['Within the past 2 years' 'Within the past year' '5 or more years ago'
 'Within the past 5 years' 'Never']
Unique Values of Age_Category Column:  ['70-74' '60-64' '75-79' '80+' '65-69' '50-54' '45-49' '18-24' '30-34'
 '55-59' '35-39' '40-44' '25-29']
Unique Values of Exercise Column:  ['No' 'Yes']
Unique Values of Heart_Disease Column:  ['No' 'Yes']
Unique Values of Skin_Cancer Column:  ['No' 'Yes']
Unique Values of Other_Cancer Column:  ['No' 'Yes']
Unique Values of Depression Column:  ['No' 'Yes']
Unique Values of Diabetes Column:  ['No' 'Yes' 'No, pre-diabetes or borderline diabetes'
 'Yes, but female told only during pregnancy']
Unique Values of Arthritis Column:  ['Yes' 'No']
Unique Values of Smoking_History Column:  ['Yes' 'No']


In [4]:
ordinal_cols = ["General_Health",
                "Checkup",
                "Age_Category",
                "Diabetes",
                "Exercise",
                "Heart_Disease",
                "Skin_Cancer",
                "Other_Cancer",
                "Depression",
                "Arthritis",
                "Smoking_History"]

ordinal_categories = [
     ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'],
     ['Never', '5 or more years ago', 'Within the past 5 years', 'Within the past 2 years', 'Within the past year'],
     ['18-24','25-29','30-34','35-39','40-44','45-49','50-54',
 '55-59','60-64','65-69','70-74','75-79','80+'],
     ["No","No, pre-diabetes or borderline diabetes","Yes, but female told only during pregnancy","Yes"],
     ["No","Yes"],
     ["No","Yes"],
     ["No","Yes"],
     ["No","Yes"],
     ["No","Yes"],
     ["No","Yes"],
     ["No","Yes"]
]

onehot_cols = ["Sex"]

In [5]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

or_enc = OrdinalEncoder(categories=ordinal_categories)
onehot_enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

transformer_pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("ordinal", or_enc, ordinal_cols),
            ("one_hot", onehot_enc, onehot_cols)
        ],
        remainder="passthrough",
        verbose_feature_names_out=False,
    )
).set_output(transform="pandas")

transformed_cvd_train = transformer_pipeline.fit_transform(cvd_data)

In [6]:
cvd_Y = transformed_cvd_train["Heart_Disease"]
cvd_X = transformed_cvd_train.drop("Heart_Disease",axis=1)

In [7]:
from sklearn.model_selection import train_test_split
cvd_train_vali_X,cvd_holdout_X, cvd_train_vali_Y,cvd_holdout_Y = train_test_split(cvd_X,cvd_Y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
scale_cols = list(cvd_train_vali_X.columns)
cols_trfrm = ColumnTransformer(transformers=[("scaler", scaler, scale_cols)],
                               remainder="passthrough",
                               verbose_feature_names_out=False)
model_lr=LogisticRegression()
model_lr_pipeline = Pipeline(steps=[("cols_transform", cols_trfrm), ("logistic", model_lr)])

param_grid_lr = {'logistic__C':np.arange(0.1,2,0.1)}
clf_lr = GridSearchCV(model_lr_pipeline, param_grid_lr, return_train_score=True, scoring="accuracy", verbose=3)
clf_lr.fit(cvd_train_vali_X, cvd_train_vali_Y)

train_scores_lr = clf_lr.cv_results_['mean_train_score']
validation_scores_lr = clf_lr.cv_results_['mean_test_score']

best_validation_score_lr = validation_scores_lr.max()
best_train_score_lr = train_scores_lr[validation_scores_lr.argmax()]
best_params_lr = clf_lr.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model_rf=RandomForestClassifier(random_state=0)
param_grid_rf = {'n_estimators':np.array([5,10,20]),
                 'max_features':np.arange(1,11,2),
                 'max_depth':np.arange(1,11,3)}
clf_rf = GridSearchCV(model_rf, param_grid_rf, return_train_score=True, scoring="accuracy", verbose=3)
clf_rf.fit(cvd_train_vali_X, cvd_train_vali_Y)

train_scores_rf = clf_rf.cv_results_['mean_train_score']
validation_scores_rf = clf_rf.cv_results_['mean_test_score']

best_validation_score_rf = validation_scores_rf.max()
best_train_score_rf = train_scores_rf[validation_scores_rf.argmax()]
best_params_rf = clf_rf.best_params_


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model_erf=ExtraTreesClassifier(random_state=0)
param_grid_erf = {'n_estimators':np.array([5,10,20]),
                 'max_features':np.arange(1,11,2),
                 'max_depth':np.arange(1,11,3)}
clf_erf = GridSearchCV(model_erf, param_grid_erf, return_train_score=True, scoring="accuracy", verbose=3)
clf_erf.fit(cvd_train_vali_X, cvd_train_vali_Y)

train_scores_erf = clf_erf.cv_results_['mean_train_score']
validation_scores_erf = clf_erf.cv_results_['mean_test_score']

best_validation_score_erf = validation_scores_erf.max()
best_train_score_erf = train_scores_erf[validation_scores_erf.argmax()]
best_params_erf = clf_erf.best_params_

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_ab=AdaBoostClassifier(random_state=0)
param_grid_ab = {'n_estimators':np.array([10,30,50,80,100]),
                 'learning_rate':np.arange(0.1,2,0.2)}
clf_ab = GridSearchCV(model_ab, param_grid_ab, return_train_score=True, scoring="accuracy", verbose=3)
clf_ab.fit(cvd_train_vali_X, cvd_train_vali_Y)

train_scores_ab = clf_ab.cv_results_['mean_train_score']
validation_scores_ab = clf_ab.cv_results_['mean_test_score']

best_validation_score_ab = validation_scores_ab.max()
best_train_score_ab = train_scores_ab[validation_scores_ab.argmax()]
best_params_ab = clf_ab.best_params_

In [10]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

model_xg=xgb.XGBClassifier()
param_grid_xg = {'n_estimators':np.array([5,10,20]),
                 'learning_rate':np.arange(0.1,2,0.2),
                 'max_depth':np.arange(1,11,3)}
clf_xg = GridSearchCV(model_xg, param_grid_xg, return_train_score=True, scoring="accuracy", verbose=3)
clf_xg.fit(cvd_train_vali_X, cvd_train_vali_Y)

train_scores_xg = clf_xg.cv_results_['mean_train_score']
validation_scores_xg = clf_xg.cv_results_['mean_test_score']

best_validation_score_xg = validation_scores_xg.max()
best_train_score_xg = train_scores_xg[validation_scores_xg.argmax()]
best_params_xg = clf_xg.best_params_

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END learning_rate=0.1, max_depth=1, n_estimators=5;, score=(train=0.919, test=0.919) total time=   0.6s
[CV 2/5] END learning_rate=0.1, max_depth=1, n_estimators=5;, score=(train=0.919, test=0.919) total time=   0.6s
[CV 3/5] END learning_rate=0.1, max_depth=1, n_estimators=5;, score=(train=0.919, test=0.919) total time=   0.6s
[CV 4/5] END learning_rate=0.1, max_depth=1, n_estimators=5;, score=(train=0.919, test=0.919) total time=   0.6s
[CV 5/5] END learning_rate=0.1, max_depth=1, n_estimators=5;, score=(train=0.919, test=0.919) total time=   0.6s
[CV 1/5] END learning_rate=0.1, max_depth=1, n_estimators=10;, score=(train=0.919, test=0.919) total time=   0.8s
[CV 2/5] END learning_rate=0.1, max_depth=1, n_estimators=10;, score=(train=0.919, test=0.919) total time=   0.8s
[CV 3/5] END learning_rate=0.1, max_depth=1, n_estimators=10;, score=(train=0.919, test=0.919) total time=   0.8s
[CV 4/5] END learning_rate=0.1

In [11]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

cvd_train2_X,cvd_erranl_X, cvd_train2_Y,cvd_erranl_Y = train_test_split(cvd_train_vali_X, cvd_train_vali_Y, test_size=0.3, random_state=0)

best_model_xg=xgb.XGBClassifier(n_estimators=best_params_xg['n_estimators'],
                                learning_rate=best_params_xg['learning_rate'],
                                max_depth=best_params_xg['max_depth'])
best_model_xg.fit(cvd_train2_X, cvd_train2_Y)

cvd_xg_predicted_Y = best_model_xg.predict(cvd_erranl_X)

error_tree = DecisionTreeClassifier(max_depth=30)
error_tree.fit(cvd_erranl_X, cvd_xg_predicted_Y)

Tot_test_samples = cvd_erranl_X.shape[0]
Tot_trai_samples = cvd_train2_X.shape[0]

cvd_errtree_predicted_Y = error_tree.predict(cvd_erranl_X)
Tot_act_test_errors = sum(~(cvd_xg_predicted_Y == cvd_erranl_Y))
Tot_pred_test_errors = sum(~(cvd_errtree_predicted_Y == cvd_erranl_Y))
print("Actual Test Errors:", Tot_act_test_errors)
print("Predicted Test Errors:", Tot_pred_test_errors)

cvd_train2_predicted_Y = error_tree.predict(cvd_train2_X)
Tot_pred_train_errors = sum(~(cvd_train2_predicted_Y == cvd_train2_Y))
print("Predicted Train Errors:", Tot_pred_train_errors)

Actual Test Errors: 5216
Predicted Test Errors: 5216
Predicted Train Errors: 12186


In [12]:
test_err_rate_errtree = np.zeros((1,error_tree.tree_.node_count))
test_err_share_errtree = np.zeros((1,error_tree.tree_.node_count))
for i in np.arange(1,cvd_erranl_X.shape[0]):
  test_err = ~(best_model_xg.predict(cvd_erranl_X.iloc[[i]]) == cvd_erranl_Y.iloc[[i]])
  test_err = test_err.iloc[0]
  test_err_rate_errtree[np.array(error_tree.decision_path(cvd_erranl_X.iloc[[i]]).toarray(), dtype=bool)] = test_err_rate_errtree[np.array(error_tree.decision_path(cvd_erranl_X.iloc[[i]]).toarray(), dtype=bool)] + test_err
test_err_share_errtree = test_err_rate_errtree/Tot_act_test_errors
test_err_rate_errtree = test_err_rate_errtree/Tot_test_samples

train_err_rate_errtree = np.zeros((1,error_tree.tree_.node_count))
for i in np.arange(1,cvd_train2_X.shape[0]):
  train_err = ~(best_model_xg.predict(cvd_train2_X.iloc[[i]]) == cvd_train2_Y.iloc[[i]])
  train_err = train_err.iloc[0]
  train_err_rate_errtree[np.array(error_tree.decision_path(cvd_train2_X.iloc[[i]]).toarray(), dtype=bool)] = train_err_rate_errtree[np.array(error_tree.decision_path(cvd_train2_X.iloc[[i]]).toarray(), dtype=bool)] + train_err
train_err_rate_errtree = train_err_rate_errtree/Tot_trai_samples


In [24]:
dot_data = export_graphviz(error_tree,
                           out_file=None,
                           max_depth=5,
                           rounded=True,
                           special_characters=True)
import pydot
dot_graph = pydot.graph_from_dot_data(dot_data)[0]

import networkx as nx
MG = nx.nx_pydot.from_pydot(dot_graph)

for n in list(MG.nodes(data=True)):
  if n[0].isnumeric():
    int_n = int(n[0])
    MG.nodes[n[0]]['label'] = f"""Test Error Share = {test_err_share_errtree[0,int_n]}
                                  \nTest Error Rate = {train_err_rate_errtree[0,int_n]}
                                  \nTrain Error Rate = {train_err_rate_errtree[0,int_n]}
                                  \n{cvd_train2_X.columns[error_tree.tree_.feature[int_n]]} ≤ {error_tree.tree_.threshold[int_n]}"""

  else:
    break

new_dot_graph = nx.nx_pydot.to_pydot(MG)
new_dot_graph.write_pdf("Error_Tree.pdf")

<ipython-input-24-7da510e6e664>:10: DeprecationWarning: nx.nx_pydot.from_pydot depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723
  MG = nx.nx_pydot.from_pydot(dot_graph)
<ipython-input-24-7da510e6e664>:23: DeprecationWarning: nx.nx_pydot.to_pydot depends on the pydot package, which hasknown issues and is not actively maintained.

See https://github.com/networkx/networkx/issues/5723
  new_dot_graph = nx.nx_pydot.to_pydot(MG)


---- Below this is the code testing area